In [1]:
import pandas as pd

In [2]:
patient_df = pd.read_csv('corona_200508.csv')
patient_df.head()

,ds,y
0,2020-01-20,1
1,2020-01-24,1
2,2020-01-26,1
3,2020-01-27,1
4,2020-01-30,1


In [3]:
patient_df.tail()

,ds,y
92,2020-05-01,8
93,2020-05-02,6
94,2020-05-03,10
95,2020-05-04,8
96,2020-05-05,3


In [4]:
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 2 columns):
ds    97 non-null object
y     97 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.6+ KB


In [5]:
# 5월 6일부터 5월 10일까지 y(해외 유입 확진자수)값은 prophet으로 예측한 값으로 설정
pred_date = pd.date_range("2020-5-6", "2020-5-10")
predict_y = [2,3,11,1,8]
predict = pd.DataFrame(data={'ds':pred_date, 'y':predict_y})
patient_df = pd.concat([patient_df, predict]).reset_index()
patient_df = patient_df.drop('index', axis=1)

In [6]:
patient_df

,ds,y
0,2020-01-20,1
1,2020-01-24,1
2,2020-01-26,1
3,2020-01-27,1
4,2020-01-30,1
...,...,...
97,2020-05-06 00:00:00,2
98,2020-05-07 00:00:00,3
99,2020-05-08 00:00:00,11
100,2020-05-09 00:00:00,1


# Feature making1 - day_cat 
- 14일 기간으로 그룹짓는 day_cat 피처 생성  

In [7]:
patient_df['day_cat'] = 0

period = 14
cat = 0

for index in list(patient_df.index):
    if index % period == 0:
        cat = cat + 1
        
    patient_df['day_cat'][index] = 'd' + str(cat)

C:\Users\eh063\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\eh063\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


# Feature making2 - y_sum, y_mean, y_std  
- day_cat 그룹끼리 확진자수인 y의 합(y_sum), 평균(y_mean), 표준편차(y_std) 피처 생성

In [8]:
patient_df

,ds,y,day_cat
0,2020-01-20,1,d1
1,2020-01-24,1,d1
2,2020-01-26,1,d1
3,2020-01-27,1,d1
4,2020-01-30,1,d1
...,...,...,...
97,2020-05-06 00:00:00,2,d7
98,2020-05-07 00:00:00,3,d8
99,2020-05-08 00:00:00,11,d8
100,2020-05-09 00:00:00,1,d8


In [9]:
features = []
f = patient_df.groupby('day_cat')['y'].agg({'y_sum':'sum'}).reset_index()
features.append(f)
f = patient_df.groupby('day_cat')['y'].agg({'y_mean':'mean'}).reset_index()
features.append(f)
f = patient_df.groupby('day_cat')['y'].agg({'y_std':'std'}).reset_index()
features.append(f)

C:\Users\eh063\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  
C:\Users\eh063\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  after removing the cwd from sys.path.
C:\Users\eh063\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  


In [10]:
for f in features :
    patient_df = pd.merge(patient_df, f, how='left',on='day_cat')

# Feature making3 - 14_ago  
- ds(날짜)에서 14일 전의 확진자 수인 14_ago 피처 생성

In [11]:
interval = 13
patient_df[str(interval+1)+'_ago'] = 0
for index in list(patient_df.index):
    if index < interval:
        patient_df[str(interval+1)+'_ago'].iloc[index] = 0
    else:
        patient_df[str(interval+1)+'_ago'].iloc[index] = patient_df['y'].iloc[index-interval]

# Feature making4 - before_sum, before_mean, before_std  
- 날짜에 해당하는 day_cat 그룹에서 이전 그룹의 sum, mean, std를 가지는 피처인 before_sum, before_mean, before_std 생성

In [12]:
patient_df['before_sum'] = 0
patient_df['before_mean'] = 0
patient_df['before_std'] = 0

for i in range(2, 9):
    index = list(patient_df[patient_df['day_cat'] == 'd'+str(i)].index)
    
    for i in index:
        patient_df['before_sum'].iloc[i] = patient_df['y_sum'].iloc[index[0]-1]
        patient_df['before_mean'].iloc[i] = patient_df['y_mean'].iloc[index[0]-1]
        patient_df['before_std'].iloc[i] = patient_df['y_std'].iloc[index[0]-1]

In [13]:
# day_cat 원-핫 인코딩 수행
patient_df = pd.get_dummies(data=patient_df, columns=['day_cat'])

In [16]:

drop_columns = ['y_sum', 'y_mean', 'y_std']
patient_df = patient_df.drop(drop_columns, axis=1)

In [17]:
patient_df

,ds,y,14_ago,before_sum,before_mean,before_std,day_cat_d1,day_cat_d2,day_cat_d3,day_cat_d4,day_cat_d5,day_cat_d6,day_cat_d7,day_cat_d8
0,2020-01-20,1,0,0,0.000000,0.000000,1,0,0,0,0,0,0,0
1,2020-01-24,1,0,0,0.000000,0.000000,1,0,0,0,0,0,0,0
2,2020-01-26,1,0,0,0.000000,0.000000,1,0,0,0,0,0,0,0
3,2020-01-27,1,0,0,0.000000,0.000000,1,0,0,0,0,0,0,0
4,2020-01-30,1,0,0,0.000000,0.000000,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2020-05-06 00:00:00,2,4,173,12.357143,7.044989,0,0,0,0,0,0,1,0
98,2020-05-07 00:00:00,3,2,84,6.000000,3.088440,0,0,0,0,0,0,0,1
99,2020-05-08 00:00:00,11,4,84,6.000000,3.088440,0,0,0,0,0,0,0,1
100,2020-05-09 00:00:00,1,9,84,6.000000,3.088440,0,0,0,0,0,0,0,1
